In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta,UTC
import math
import time

In [2]:
   # pause between batches

In [4]:
nifty500_csv="/Users/a0s0iro/PycharmProjects/Stock-Analysis-agent/files/ind_nifty500list.csv"
const = pd.read_csv(nifty500_csv)
symbols = const["Symbol"].dropna().astype(str).str.strip().unique().tolist()

tickers = [s + ".NS" for s in symbols]
# tickers = ["3MINDIA.NS","360ONE.NS","ABB.NS","TATACHEM.NS","ADANIENT.NS","POWERINDIA.NS","SBILIFE.NS"]
tickers = ['ACMESOLAR.NS', 'AIAENG.NS', 'APLAPOLLO.NS', 'AUBANK.NS']


In [8]:
def fetch_price_data(ticker, period="6mo", interval="1d"):
    """
    Fetch OHLCV data using yfinance.
    """
    df = yf.download(ticker, period=period, interval=interval)
    df = df.dropna()
    return df

In [42]:
# def compute_rsi(close_prices, period=14):
#     delta = close_prices.diff()
#     gain = delta.clip(lower=0)
#     loss = -delta.clip(upper=0)

#     avg_gain = gain.rolling(period).mean()
#     avg_loss = loss.rolling(period).mean()

#     rs = avg_gain / avg_loss
#     rsi = 100 - (100 / (1 + rs))
#     return rsi.iloc[-1]
def rma(series, period):
    return series.ewm(alpha=1/period, adjust=False).mean()

def compute_rsi(close_prices, period=14):
    close = pd.Series(np.array(close_prices).reshape(-1)).astype(float)

    delta = close.diff()

    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = rma(gain, period)
    avg_loss = rma(loss, period)

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return float(rsi.iloc[-1])

In [59]:
def compute_bollinger_bands(close_prices, length=20, std_dev=2, squeeze_threshold=0.025):
    # Ensure 1D input
    close = pd.Series(np.array(close_prices).reshape(-1)).astype(float)

    # Middle band = SMA
    middle = close.rolling(length).mean()

    # Standard deviation
    std = close.rolling(length).std()

    # Upper & lower bands
    upper = middle + std_dev * std
    lower = middle - std_dev * std

    # Latest values (convert to float)
    upper_val = float(upper.iloc[-1])
    middle_val = float(middle.iloc[-1])
    lower_val = float(lower.iloc[-1])

    # Band width % relative to price
    bandwidth = (upper_val - lower_val) / middle_val

    # ----------------------
    # SQUEEZE DETECTION
    # ----------------------
    squeeze = bandwidth < squeeze_threshold

    # Expansion: Squeeze ended recently and now bandwidth increasing
    prev_bandwidth = (upper.iloc[-2] - lower.iloc[-2]) / middle.iloc[-2]
    expansion = (prev_bandwidth < squeeze_threshold) and (bandwidth > prev_bandwidth)

    # Return structured data
    return {
        "upper_band": upper_val,
        "middle_band": middle_val,
        "lower_band": lower_val,
        "bandwidth": float(bandwidth),

        "squeeze": bool(squeeze),       # low volatility
        "expansion": bool(expansion),   # volatility breakout

        "previous_upper": float(upper.iloc[-2]),
        "previous_middle": float(middle.iloc[-2]),
        "previous_lower": float(lower.iloc[-2]),
        "previous_bandwidth": float(prev_bandwidth)
    }


In [46]:
def rma(series, period):
    return series.ewm(alpha=1/period, adjust=False).mean()

def compute_adx(high, low, close, period=14):
    # FORCE inputs to 1-D arrays → fixes your error
    high = pd.Series(np.array(high).reshape(-1)).astype(float)
    low = pd.Series(np.array(low).reshape(-1)).astype(float)
    close = pd.Series(np.array(close).reshape(-1)).astype(float)

    # True Range
    prev_close = close.shift(1)
    tr = pd.concat([
        (high - low),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)

    # Directional Movement
    up_move = high - high.shift(1)
    down_move = low.shift(1) - low

    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0)

    # Wilder smoothing (TradingView style)
    tr_rma = rma(tr, period)
    plus_dm_rma = rma(pd.Series(plus_dm), period)
    minus_dm_rma = rma(pd.Series(minus_dm), period)

    plus_di = 100 * (plus_dm_rma / tr_rma)
    minus_di = 100 * (minus_dm_rma / tr_rma)

    dx = 100 * abs(plus_di - minus_di) / (plus_di + minus_di)

    adx = rma(dx, period)  # ADX = RMA of DX

    return float(adx.dropna().iloc[-1])


In [27]:
def compute_ema(close_prices, period=20):
    return close_prices.ewm(span=period, adjust=False).mean().iloc[-1]

In [51]:
def compute_macd(close_prices, fast=12, slow=26, signal=9):
    # Ensure 1D input
    close = pd.Series(np.array(close_prices).reshape(-1)).astype(float)

    # TradingView EMA (standard EMA)
    ema_fast = close.ewm(span=fast, adjust=False).mean()
    ema_slow = close.ewm(span=slow, adjust=False).mean()

    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    histogram = macd_line - signal_line

    # Extract numeric values
    prev_macd = float(macd_line.iloc[-2])
    prev_signal = float(signal_line.iloc[-2])
    prev_hist = float(histogram.iloc[-2])

    curr_macd = float(macd_line.iloc[-1])
    curr_signal = float(signal_line.iloc[-1])
    curr_hist = float(histogram.iloc[-1])

    # --------------------------
    # TREND (Ongoing)
    # --------------------------
    if curr_macd > curr_signal:
        trend = "bullish"
    else:
        trend = "bearish"

    # --------------------------
    # CROSSOVER (Event)
    # --------------------------
    if prev_macd < prev_signal and curr_macd > curr_signal:
        crossover = "bullish"
    elif prev_macd > prev_signal and curr_macd < curr_signal:
        crossover = "bearish"
    else:
        crossover = "none"

    # --------------------------
    # HISTOGRAM MOMENTUM (Strength)
    # --------------------------
    if curr_hist > prev_hist:
        momentum = "increasing"
    elif curr_hist < prev_hist:
        momentum = "decreasing"
    else:
        momentum = "flat"

    # --------------------------
    # RETURN STRUCTURED OUTPUT
    # --------------------------
    return {
        "macd_value": curr_macd,
        "signal_value": curr_signal,
        "histogram_value": curr_hist,

        # Trend classification:
        "trend": trend,                 # bullish or bearish

        # Crossover classification:
        "crossover": crossover,         # bullish / bearish / none

        # Momentum classification:
        "momentum": momentum,           # increasing / decreasing / flat

        # Raw previous values (agent can infer patterns)
        "previous_macd": prev_macd,
        "previous_signal": prev_signal,
        "previous_histogram": prev_hist
    }


In [34]:
def compute_volume_indicator(volume):
    # Force volume into 1D Series
    volume = pd.Series(np.array(volume).reshape(-1))

    avg_20 = volume.rolling(20).mean()

    last_volume = float(volume.iloc[-1])
    avg_20_volume = float(avg_20.iloc[-1])

    trend = "High" if last_volume > avg_20_volume else "Low"

    return {
        "last_volume": last_volume,
        "avg_20_volume": avg_20_volume,
        "volume_trend": trend
    }


In [57]:
def get_indicator(ticker, indicator, period=14, yf_period="6mo", yf_interval="1d"):
    """
    Main tool function your agent will call.
    Returns any technical indicator for a ticker.
    """

    df = fetch_price_data(ticker, period=yf_period, interval=yf_interval)
  

    indicator = indicator.upper()

    if indicator == "RSI":
        return {"indicator": "RSI", "value": float(compute_rsi(df["Close"], period))}
    
    elif indicator == "ADX":
        return {"indicator": "ADX", "value": float(compute_adx(df["High"], df["Low"], df["Close"], period))}

    elif indicator == "EMA":
        return {"indicator": "EMA", "value": float(compute_ema(df["Close"], period))}

    elif indicator == "MACD":
        return {"indicator": "MACD", **compute_macd(df["Close"])}

    elif indicator == "VOLUME":
        return {"indicator": "VOLUME", **compute_volume_indicator(df["Volume"])}
    elif indicator=="BOLLINGER_BAND":
        return {"indicator":"BOLLINGER_BAND",**compute_bollinger_bands(df["Close"])}

    else:
        return {"error": f"Unknown indicator requested: {indicator}"}

In [36]:
def fetch_price_data(ticker, period="6mo", interval="1d"):
    """
    Fetch OHLCV data using yfinance.
    """
    df = yf.download(ticker, period=period, interval=interval)
    df = df.dropna()
    return df

In [37]:
df = fetch_price_data("APLAPOLLO.NS", period="1mo", interval="1d")


/var/folders/1b/4c3w60fs43d4gkd3fr4t51pr0000gp/T/ipykernel_51814/4104416519.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed


In [38]:
df

Price,Close,High,Low,Open,Volume
Ticker,APLAPOLLO.NS,APLAPOLLO.NS,APLAPOLLO.NS,APLAPOLLO.NS,APLAPOLLO.NS
Date,,,,,
2025-11-12,1796.400024,1806.900024,1778.500000,1806.900024,630889
2025-11-13,1765.400024,1802.500000,1760.800049,1798.400024,528818
2025-11-14,1763.300049,1777.800049,1737.000000,1769.000000,273177
2025-11-17,1777.699951,1787.599976,1763.300049,1763.300049,226118
2025-11-18,1763.800049,1782.800049,1757.199951,1778.000000,244549
2025-11-19,1720.900024,1770.599976,1717.000000,1754.000000,541702
2025-11-20,1721.199951,1746.099976,1714.000000,1730.000000,1063781
2025-11-21,1727.800049,1732.699951,1703.000000,1727.000000,450086


In [39]:
df["High"]

Ticker,APLAPOLLO.NS
Date,
2025-11-12,1806.900024
2025-11-13,1802.500000
2025-11-14,1777.800049
2025-11-17,1787.599976
2025-11-18,1782.800049
2025-11-19,1770.599976
2025-11-20,1746.099976
2025-11-21,1732.699951
2025-11-24,1732.699951


In [61]:
get_indicator("SBIN.NS","BOLLINGER_BAND")

/var/folders/1b/4c3w60fs43d4gkd3fr4t51pr0000gp/T/ipykernel_51814/4104416519.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed


{'indicator': 'BOLLINGER_BAND',
 'upper_band': 990.4610521919599,
 'middle_band': 969.2799926757813,
 'lower_band': 948.0989331596027,
 'bandwidth': 0.043704728615529265,
 'squeeze': False,
 'expansion': False,
 'previous_upper': 990.5131909167852,
 'previous_middle': 969.514990234375,
 'previous_lower': 948.5167895519647,
 'previous_bandwidth': 0.04331691803410702}

In [17]:
get_indicator("APLAPOLLO.NS","EMA")

/var/folders/1b/4c3w60fs43d4gkd3fr4t51pr0000gp/T/ipykernel_51814/4104416519.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed
/var/folders/1b/4c3w60fs43d4gkd3fr4t51pr0000gp/T/ipykernel_51814/486959844.py:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return {"indicator": "EMA", "value": float(compute_ema(df["Close"], period))}


{'indicator': 'EMA', 'value': 1740.9753903290054}

In [52]:
get_indicator("SBIN.NS","MACD")

/var/folders/1b/4c3w60fs43d4gkd3fr4t51pr0000gp/T/ipykernel_51814/4104416519.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed


{'indicator': 'MACD',
 'macd_value': 5.288155980321108,
 'signal_value': 8.958541066464617,
 'histogram_value': -3.6703850861435097,
 'trend': 'bearish',
 'crossover': 'none',
 'momentum': 'increasing',
 'previous_macd': 5.724700346848749,
 'previous_signal': 9.876137338000495,
 'previous_histogram': -4.151436991151746}

In [47]:
get_indicator("APLAPOLLO.NS","ADX")

/var/folders/1b/4c3w60fs43d4gkd3fr4t51pr0000gp/T/ipykernel_51814/4104416519.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed


{'indicator': 'ADX', 'value': 19.215971502416956}

In [35]:
get_indicator("APLAPOLLO.NS","VOLUME")

/var/folders/1b/4c3w60fs43d4gkd3fr4t51pr0000gp/T/ipykernel_51814/4104416519.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed


{'indicator': 'VOLUME',
 'last_volume': 555246.0,
 'avg_20_volume': 441666.95,
 'volume_trend': 'High'}